In [98]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

import os
from PIL import Image


In [99]:
# Define a simple feedforward neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class FCN(nn.Module):
    "Defines a connected network"
    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x
    
def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=int(1000/fps), loop=loop)

In [100]:
def neural_net_solver(x_data, y_data, x, y):
    # Instantiate the network, define a loss function and an optimizer
    net = Net()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    # Convert the training data to PyTorch tensors
    x_train = torch.tensor(x_data, dtype=torch.float32)
    y_train = torch.tensor(y_data, dtype=torch.float32)

    # Train the network
    for epoch in range(5000):
        optimizer.zero_grad()
        outputs = net(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    # Predict the y-values over the full x-range
    x_full = torch.tensor(x, dtype=torch.float32)
    y_pred = net(x_full)

    # Plot the results
    plt.figure(figsize=(10,6))
    plt.plot(x.numpy(), y.numpy(), label='Analytical solution')
    plt.plot(x_data.numpy(), y_data.numpy(), 'o', label='Training data')
    plt.plot(x.numpy(), y_pred.detach().numpy(), '--', label='ML prediction')
    plt.legend()
    plt.show()

In [101]:
def linear_regressor_solver(x_data, y_data, x, y):
    # Reshape the data as sklearn expects 2D input
    x_data_np = x_data.numpy().reshape(-1, 1)
    y_data_np = y_data.numpy().reshape(-1, 1)
    x_np = x.numpy().reshape(-1, 1)

    reg = LinearRegression().fit(x_data_np, y_data_np)
    y_pred = reg.predict(x_np)

    plt.plot(x.numpy(), y.numpy(), label='Analytical solution')
    plt.plot(x_data_np, y_data_np, 'o', label='Training data')
    plt.plot(x_np, y_pred, '--', label='Linear Regression prediction')
    plt.legend()
    plt.show()

In [102]:
def support_vector_solver(x_data, y_data, x, y):
    x_data_np = x_data.numpy().reshape(-1, 1)
    y_data_np = y_data.numpy().reshape(-1, 1)
    x_np = x.numpy().reshape(-1, 1)
    
    svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
    svr.fit(x_data_np, y_data_np.ravel())
    y_pred = svr.predict(x_np)

    plt.plot(x.numpy(), y.numpy(), label='Analytical solution')
    plt.plot(x_data_np, y_data_np, 'o', label='Training data')
    plt.plot(x_np, y_pred, '--', label='SVR prediction')
    plt.legend()
    plt.show()

In [103]:
def decision_tree_solver(x_data, y_data, x, y):
    x_data_np = x_data.numpy().reshape(-1, 1)
    y_data_np = y_data.numpy().reshape(-1, 1)
    x_np = x.numpy().reshape(-1, 1)
    
    tree = DecisionTreeRegressor(max_depth=5)
    tree.fit(x_data_np, y_data_np.ravel())
    y_pred = tree.predict(x_np)

    plt.plot(x.numpy(), y.numpy(), label='Analytical solution')
    plt.plot(x_data_np, y_data_np, 'o', label='Training data')
    plt.plot(x_np, y_pred, '--', label='Decision Tree prediction')
    plt.legend()
    plt.show()

In [104]:
def Gaussian_process_solver(x_data, y_data, x, y):
    x_data_np = x_data.numpy()
    y_data_np = y_data.numpy()

    # Kernel with initial parameters
    kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))

    # Create a GP model
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1)

    # Fit to data using Maximum Likelihood Estimation of the parameters
    gp.fit(x_data_np, y_data_np)

    # Make the prediction on the meshed x-axis (ask for Standard Deviation as well)
    x_np = x.numpy()
    y_pred, sigma = gp.predict(x_np, return_std=True)

    # Plot the function, the prediction and the 95% confidence interval
    plt.figure()
    plt.plot(x,y, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
    plt.plot(x_np, y_pred, 'b-', label='GP prediction')
    plt.fill_between(x_np.flatten(), y_pred.flatten() - sigma, y_pred.flatten() + sigma, alpha=0.2, color='b')
    plt.scatter(x_data_np, y_data_np, c='r', label='Training data')
    plt.xlabel('$x$')
    plt.ylabel('$f(x)$')
    plt.legend(loc='upper left')
    plt.show()

In [105]:
def optimized_nn_solver(x_data, y_data, x, y):
    def plot_result(x,y,x_data,y_data,yh,xp=None):
        "Pretty plot training results"
        plt.figure(figsize=(8,4))
        plt.plot(x,y, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
        plt.plot(x,yh, color="tab:blue", linewidth=4, alpha=0.8, label="Neural network prediction")
        plt.scatter(x_data, y_data, s=60, color="tab:orange", alpha=0.4, label='Training data')
        if xp is not None:
            plt.scatter(xp, -0*torch.ones_like(xp), s=60, color="tab:green", alpha=0.4, 
                        label='Physics loss training locations')
        l = plt.legend(loc=(1.01,0.34), frameon=False, fontsize="large")
        plt.setp(l.get_texts(), color="k")
        plt.xlim(-0.05, 1.05)
        plt.ylim(-1.1, 1.1)
        plt.text(1.065,0.7,"Training step: %i"%(i+1),fontsize="xx-large",color="k")
        plt.axis("off")
    
    
    # train standard neural network to fit training data
    torch.manual_seed(123)
    model = FCN(1,1,32,3)
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    files = []
    plot_dir = 'nn_plots'
    if not os.path.exists('nn_plots'):
        os.makedirs('nn_plots')

    for i in range(1000):
        optimizer.zero_grad()
        yh = model(x_data)
        loss = torch.mean((yh-y_data)**2)# use mean squared error
        loss.backward()
        optimizer.step()
    
    
        # plot the result as training progresses
        if (i+1) % 10 == 0: 
        
            yh = model(x).detach()
        
            plot_result(x,y,x_data,y_data,yh)
        
            file = "nn_plots/nn_%.8i.png"%(i+1)
            plt.savefig(file, bbox_inches='tight', pad_inches=0.1, dpi=100, facecolor="white")
            files.append(file)
    
            if (i+1) % 500 == 0: plt.show()
            else: plt.close("all")

    return files